In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import os
from torchvision import transforms
import pandas as pd
import matplotlib.pyplot as plt
import torch.optim as optim


from dataset import Thyroid_Dataset
from model import Eff_Unet
from loss_metric import DiceLoss, IOU_score

C:\Users\oplab\anaconda3\envs\thyroid\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
C:\Users\oplab\anaconda3\envs\thyroid\Lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


In [22]:
image_size = 128
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])
train_dataset = Thyroid_Dataset("train.csv", transform = transform)
train_dataloader = DataLoader(train_dataset, batch_size = 1)
image, mask = next(iter(train_dataloader))
image = image.cuda()
mask = mask.cuda()

In [75]:
model = Eff_Unet(
        layers=[5, 5, 15, 10],
        embed_dims=[40, 80, 192, 384],
        downsamples=[True, True, True, True],
        vit_num=6,
        drop_path_rate=0.1,
        num_classes=1,
        resolution = image_size).cuda()


In [76]:
loss_fn = DiceLoss()

In [77]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [78]:
for epoch in range(1000):
    outputs = model(image)
    # outputs = torch.sigmoid(logits)
    loss = loss_fn(outputs, mask)
    IOU = IOU_score(outputs, mask)

    # Backward and optimize
    optimizer.zero_grad()   # clear previous gradients
    loss.backward()         # compute gradients
    optimizer.step()        # update weights

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, IOU: {IOU.item():.4f}")

Epoch 1, Loss: 0.9167, IOU: 0.0337
Epoch 2, Loss: 0.9054, IOU: 0.0759
Epoch 3, Loss: 0.8858, IOU: 0.1140
Epoch 4, Loss: 0.8637, IOU: 0.1657
Epoch 5, Loss: 0.8474, IOU: 0.2217
Epoch 6, Loss: 0.8353, IOU: 0.3014
Epoch 7, Loss: 0.8266, IOU: 0.3790
Epoch 8, Loss: 0.8196, IOU: 0.3988
Epoch 9, Loss: 0.8143, IOU: 0.4180
Epoch 10, Loss: 0.8100, IOU: 0.4525
Epoch 11, Loss: 0.8061, IOU: 0.4769
Epoch 12, Loss: 0.8025, IOU: 0.4904
Epoch 13, Loss: 0.7990, IOU: 0.5154
Epoch 14, Loss: 0.7958, IOU: 0.5562
Epoch 15, Loss: 0.7927, IOU: 0.5647
Epoch 16, Loss: 0.7897, IOU: 0.5899
Epoch 17, Loss: 0.7868, IOU: 0.6083
Epoch 18, Loss: 0.7840, IOU: 0.6150
Epoch 19, Loss: 0.7812, IOU: 0.6294
Epoch 20, Loss: 0.7785, IOU: 0.6440
Epoch 21, Loss: 0.7757, IOU: 0.6494
Epoch 22, Loss: 0.7729, IOU: 0.6554
Epoch 23, Loss: 0.7702, IOU: 0.6626
Epoch 24, Loss: 0.7674, IOU: 0.6706
Epoch 25, Loss: 0.7646, IOU: 0.6841
Epoch 26, Loss: 0.7618, IOU: 0.6895
Epoch 27, Loss: 0.7590, IOU: 0.6908
Epoch 28, Loss: 0.7562, IOU: 0.7045
E

KeyboardInterrupt: 